## 1. 任务栈是什么
- 任务栈 Task，是一种用来放置 Activity 实例的容器，它以栈的形式进行盛放，也就是所谓的先进后出，主要有2个基本操作：压栈和出栈。其所存放的Activity 是不支持重新排序的，只能根据压栈和出栈操作更改 Activity 的顺序。
- 启动一个 Application 的时候，系统会为它默认创建一个对应的 Task，用来放置根 Activity。默认启动 Activity 会放在同一个Task中，新启动的Activity 会被压入启动它的那个 Activity 的栈中，并且显示它。当用户按下回退键时，这个 Activity 就会被弹出栈，按下 Home 键回到桌面，再启动另一个应用，这时候之前那个 Task 就被移到后台，成为后台任务栈，而刚启动的那个 Task 就被调到前台，成为前台任务栈，Android 系统显示的就是前台任务栈中的 Top 实例 Activity。

---

## 2. 任务栈的作用
- 由于 android 强化了组件概念，弱化了 Aplication 的概念，所以在 android 程序开发中，A应用的A组件想要使用拍照或录像的功能就可以不用去针对 Camera 类进行开发，直接调用系统自带的摄像头应用（称其B应用）中的组件（称其B组件）就可以了。但是这就引发了一个新问题，A 组件运行在 A 应用中，B 组件运行在 B 应用中，自然都不在同一个进程中，那么从 B 组件中返回的时候，如何实现正确返回到 A 组件呢？
- Task 就是来负责实现这个功能的，它是从用户角度来理解应用而建立的一个抽象概念。因为用户所能看到的组件就是 Activity，所以 Task 可以理解为实现一个功能而负责管理所有用到的 Activity 实例的栈。

---

## 3. 启动模式
### 3.1 standard
- 默认模式，可以不用写配置
- 在这个模式下，都会默认创建一个新的实例
- 在这种模式下，可以有多个相同的实例，也允许多个相同 Activity 叠加
- 如果以这种方式启动的 Activity 被跨进程调用，在 5.0 之前新启动的 Activity 实例会放入发送 Intent 的 Task 的栈的顶部，尽管它们属于不同的程序，这似乎有点费解看起来也不是那么合理，所以在 5.0 之后，上述情景会创建一个新的 Task，新启动的 Activity 就会放入刚创建的 Task 中，这样更合理。

### 3.2 singleTop
- 栈顶复用模式，如果要开启的 activity 在任务栈的顶部已经存在，就不会创建新的实例，而是调用 onNewIntent() 方法。避免栈顶的 activity 被重复的创建。
- 应用场景：在通知栏点击收到的通知，然后需要启动一个 Activity，这个 Activity 就可以用 singleTop，否则每次点击都会新建一个 Activity
- 如果是外部程序启动 singleTop 的 Activity，在 Android 5.0 之前新创建的 Activity 会位于调用者的 Task 中，5.0 及以后会放入新的 Task 中。

### 3.3 singleTask
- 栈内复用模式，activity 只会在任务栈里面存在一个实例。如果要激活的 activity，在任务栈里面已经存在，就不会创建新的 activity，而是复用这个已经存在的 activity，调用 onNewIntent() 方法，并且清空这个 activity 任务栈上面所有的 activity。
- 应用场景：大多数 App 的主页。
- 对于大部分应用，当我们在主界面点击回退按钮的时候都是退出应用，那么当我们第一次进入主界面之后，主界面位于栈底，以后不管我们打开了多少个 Activity，只要我们再次回到主界面，都应该使用将主界面 Activity 上所有的 Activity 移除的方式来让主界面 Activity 处于栈顶，而不是往栈顶新加一个主界面 Activity 的实例，通过这种方式能够保证退出应用时所有的 Activity 都能报销毁。

- 在跨应用 Intent 传递时，如果系统中不存在 singleTask Activity 的实例，那么将创建一个新的 Task，然后创建 SingleTask Activity 的实例，将其放入新的 Task 中。

#### 场景1
- 假如目前有个任务栈 T1 中的情况是 ABC，这个时候 Activity D 以 singleTask 模式请求启动，其所需要的任务栈正是 T1，则系统会直接创建 D 的实例并将其入栈到 T1 中

#### 场景2
- 假如 Activity D 启动所需要的任务栈为 T2, 由于 T2 和 D 的实例均不存在，那么系统会先创建任务栈 T2，然后再创建 D 的实例并将其入栈到 T2 中。我们可以通过设置 Activity 的 taskAffinity 属性来模拟这一场景。

```
<activity 
    android:name=".SingleTaskActivity" 
    android:label="singleTask launchMode" 
    android:launchMode="singleTask" 
    android:taskAffinity="">
</activity>
```

#### 场景3
- 如果 Activity D 所需的任务栈为 T3，并且当前任务栈 T3 的情况为 ADBC，根据栈内复用的原则，此时 D 不会重新创建，系统会把 D 切换到栈顶并调用其 onNewIntent() 方法，同时由于 singleTask 默认具有 ClearTop 的效果，会导致栈内所有在 D 上面的 Activity 全部出栈，于是最终 T3 的情况为 AD。

#### 场景4
- 假如目前有两个任务栈，前台任务栈 T4 的情况为 AB, 后台任务栈 t4 里存有 CD, 假设 CD 的启动模式均为 singleTask，现在由 B 去启动 D, 那么整个后台任务都会被切换到前台，这个时候整个栈就变成了 ABCD。

#### 场景5
- 假如场景4的其他条件不变，B 启动的是 C 而不是 D, 那么整个栈的情况就变成了 ABC, 因为 D 在 C 上面，会被清理出栈。

### 3.4 singleInstance
- 单一实例模式，整个手机操作系统里面只有一个实例存在。不同的应用去打开这个 activity 共享公用的同一个 activity。它会运行在自己单独，独立的任务栈里面，并且任务栈里面只有他一个实例存在。
- 应用场景：呼叫来电界面。

---

## 4. 设置 Intent 的 Flag
- 系统提供了两种方式来设置一个 Activity 的启动模式，除了在 AndroidManifest 文件中设置以外，还可以通过 Intent 的 Flag 来设置一个Activity 的启动模式

### 4.1 FLAG_ACTIVITY_NEW_TASK
- 使用一个新的 Task 来启动一个 Activity，但启动的每个 Activity 都将在一个新的 Task 中。该 Flag 通常使用在从 Service 中启动 Activity 的场景，由于 Service 中并不存在 Activity 栈，所以使用该 Flag 来创建一个新的 Activity 栈，并创建新的 Activity 实例。

### 4.2 FLAG_ACTIVITY_SINGLE_TOP
- 使用 Singletop 模式启动一个 Activity，与指定 android:launchMode=“singleTop” 效果相同

### 4.3 FLAG_ACTIVITY_CLEAR_TOP
- 使用 SingleTask 模式来启动一个 Activity，与指定 android:launchMode=“singleTask” 效果相同

### 4.4 FLAG_ACTIVITY_NO_HISTORY
- Activity 使用这种模式启动 Activity，当该 Activity 启动其他 Activity 后，该 Activity 就消失了，不会保留在 Activity 栈中。
- 应用场景：跳板页？

### 4.5 LaunchMode 与 StartActivityForResult
- 开发中经常使用 StartActivityForResult 方式启动一个 Activity，并在 onActivityResult 方法中做处理。但你有可能遇到过拿不到返回值的情况，那有可能是因为 Activity 的 LaunchMode 设置为了 singleTask。
- 示例：两个 Activity，A 和 B，现在由 A 页面跳转到 B 页面，看一下 LaunchMode 与 StartActivityForResult 之间的关系

![image](page1.png)

![image](page2.png)
 
- 在 5.0 之前，当启动一个 Activity 时，系统将首先检查 Activity 的 launchMode，如果为 A 页面设置为 SingleInstance 或者 B 页面设置为 singleTask 或者 singleInstance, 则会在 LaunchFlags 中加入 FLAG_ACTIVITY_NEW_TASK 标志，而如果含有 FLAG_ACTIVITY_NEW_TASK 标志的话，onActivityResult 将会立即接收到一个 cancel 的信息，而 5.0 之后这个方法做了修改，onActivityResult 依旧可以正常工作。

---

## 5. onNewIntent 调用时机
![image](page3.png)

- 当 activity (假设为 A) 的 launchMode 为 singleTop 且 A 的实例已经在 task 栈顶，或者 launchMode 为 singleTask 且 A 的实例已在 task 栈里 (无论是栈顶还是栈中)，再次启动 activity A 时，便不会调用 onCreate() 去产生新的实例，而是调用 onNewIntent() 并重用 task 栈里的 A 实例。
- 如果 A 在栈顶，那么调用顺序依次是 A.onPause() –> A.onNewIntent() –> A.onResume()。A 的 launchMode 可以是 singleTop 或者是 singlTask。
- 如果 A 不在栈顶，此时它处于 A.onStop() 状态，当再次启动时，调用顺序依次是 (A.onStop()) –> A.onNewIntent() –> A.onRestart() –> A.onStart() –> A.onResume()。A 的 launchMode 只能是 singleTask。

### 补充：两个 Activity 调用的生命周期
![image](page4.png)

---